In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt, style

style.use('ggplot')
%pylab inline

In [ ]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

### Анализ имеющихся  признаков

#### Для начала посмотрим что из себя представляет датасет:

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df_test.describe()

In [ ]:
df.info()

In [ ]:
df_test.info()

Из статистики в тренировочном и тестовом детасетах становится понятно, что есть опреденные проблемы с категориями Age, Fare и Cabin, где есть отсутствующие значения, при анализе надо это учитывать. 

In [ ]:
df.head(10)

<ul>
<li>PassengerId - Идентификатор пассажира. Вряд ли от него можно получить какую-то пользу.</li>
<li>Survived - Целевая переменная выжил/невыжил, которую надо найти - бинарный признак.</li>
<li>Pclass - Пассажирский класс - количественный признак (1 - первый; 2 - второй; 3 - третий).</li>
<li>Name - Имя пассажира - категориальный признак.</li>
<li>Sex - Пол пассажира - категориальный признак, который можно преобразовать в бинарный.</li>
<li>Age - Возраст пассажира - количественный признак - количественный признак.</li>
<li>SibSp - Количество братьев/сестер на борту коробля - количественный признак.</li>
<li>Parch - Количество родителей/детей на борту коробля - количественный признак.</li>
<li>Ticket - Номер билета - категориальный признак.</li>
<li>Fare - Цена билета - количественный признак.</li>
<li>Cabin - Номера каюты - категориальный признак.</li>
<li>Embarked - Место посадки (C - Cherbourg; Q - Queenstown; S - Southampton) - категориальный признак.</li>
</ul>

### Проведем небольшой анализ данных, чтобы понять какие нужны и как их лучше преобразовать

Для начала надо подумать, что делать с пропущенными значениями, думаю, что самый оптимальный вариант, это заполнить их медианными значениями для количественных признаков.

In [ ]:
df = df.fillna(df.median(axis=0), axis=0)

#### Для начала посмотрим распреденение погибших и выживших:

In [ ]:
df['Survived'].value_counts(normalize=True)

#### Теперь посмотрим  распреденение погибших и выживших по полу, для этого преобразуем категориальный признак в количественный:

In [ ]:
df['SexCode'] = df['Sex'].map(lambda sex: 1 if sex == 'male' else 0)
df.head(5)

In [ ]:
df.groupby(['SexCode', 'Survived']).size().unstack().plot(kind='bar', stacked=True);

Как видно, что в процентном соотношении, женщин погибло гораздо меньше чем мужчин. Возьмем этот факт на заметку.

#### Теперь посмотрим распреденение по классу:

In [ ]:
df.groupby(['Pclass', 'Survived']).size().unstack().plot(kind='bar', stacked=True);

Видно, что в 1-ом (самом дорогом и самом близким к палубе) классе выживших больше всего, тогда как в 3-м ситуация ровно противоположная.

#### Проверим, стоит ли использовать категорию Fare или лучше ориентироваться на класс:

In [ ]:
df.groupby('Pclass')['Fare'].mean().plot(kind='bar');

In [ ]:
df.groupby('Pclass')['Fare'].max()

In [ ]:
df[df['Fare'] > 0].groupby('Pclass')['Fare'].min()

Несмотря на то, что в среднем билеты 1-го класса стоят намного дороже чем 2-го и 3-го, но наблдается большая разница между самым дорогим и самым дешевым билетом, особенно в 1-ом классе. Пока непонятно что делать с этим признаком.

#### Теперь посмотрим на влияние наличия/отсутствия родственников:

In [ ]:
df.groupby(['SibSp', 'Survived']).size().unstack().plot(kind='bar', stacked=True);

In [ ]:
df.groupby(['Parch', 'Survived']).size().unstack().plot(kind='bar', stacked=True);

Как видно, наличие хотя бы одного родственника на борту несколько повышает шанс на спасение. Попробуем добвить еще и пол к этим графикам.

In [ ]:
df.groupby(['SibSp', 'SexCode', 'Survived']).size().unstack().plot(kind='bar', stacked=True);

In [ ]:
df.groupby(['Parch', 'SexCode', 'Survived']).size().unstack().plot(kind='bar', stacked=True);

Как видно, наличие родственников на борту лучше сказывается на женщинах, чем мужчинах. Поробуем установить родственные связи, для этого выделим фамилию в отдельный признак и посмотрим, что получится.

In [ ]:
df['Surname'] = df['Name'].map(lambda name: name.split(',')[0].strip())
df[(df['SibSp'] > 0) | (df['Parch'] > 0)].sort_values(['Surname']).head(15)

Можно заметить, что наличие родственников можно определить не только по фамилий, но и (вполне возможно), что и по единому номеру билета. С другой стороны, не для всех пассажиров указаны все родственники (как для фамилии Aks или Ahlin), возможно это связано с разделением на тестовую и тренировочную выборки, поэтому это не может являться надежной категорией для исследования. С другой стороны, наличия родителя Parch и возраст, скажем, до 16-ти лет, может довольно четко дать понять, что это ребенок с родителями. Проверим это предположение.

In [ ]:
df[(df['Parch'] > 0) & (df['Age'] <= 18)]['Survived'].value_counts(normalize=True)

In [ ]:
df[df['Age'] <= 18]['Survived'].value_counts(normalize=True)

In [ ]:
df[(df['Parch'] > 0) & (df['Age'] <= 18)] \
    .groupby(['Age', 'Survived']).size().unstack().plot(kind='bar', stacked=True);

In [ ]:
df[df['Age'] <= 18] \
    .groupby(['Age', 'Survived']).size().unstack().plot(kind='bar', stacked=True);

Как видно, быть ребенком с родителями сильно увеличивает шанс на спасение по сравнению с "рядовым" пассажиром, поэтому выделим это в отдельный признак.

In [ ]:
df['IsChild'] = df.apply(lambda row: 1 if row['Age'] <= 18 and row['Parch'] > 0 else 0, axis=1)
df.head(10)

In [ ]:
df[df['Parch'] > 0]['Survived'].value_counts(normalize=True)

Как можно заметить, наличие родителей или детей в определенной степени также повышает вероятность выжить, поэтому будем учитывать и этот признак тоже.

#### Раз уж затронули тему возраста, то исследуем еще и распределение по нему:

In [ ]:
df['Age'].plot(kind='hist')

In [ ]:
df['Age'].mean()

Так как напрямую работать с возрастом не очень удобно, то разобьем его на группы. А отсутствующие значения заменим нулями.

In [ ]:
df['AgeGroup'] = df['Age']
df['AgeGroup'] = df['AgeGroup'].map(lambda age: int(age // 10) + 1)
df.head(10)

In [ ]:
df['AgeGroup'].value_counts()

Получились следующие группы 0 - для тех у кого возраст не указан, 1 - возраст от 0 до 9 лет, 2 - от 10 до 19 и т.д. Из них - самая многочисленная группа 3 - от 20 до 29 лет.

In [1]:
from sklearn.preprocessing import OneHotEncoder

#### В итоге, после краткого анализа можно определиться со следующими признаками, которые могут оказать влияние на качество обучения:

<ul>
<li>Pclass - Пассажирский класс - количественный признак (1 - первый; 2 - второй; 3 - третий).</li>
<li>SexCode - Пол пассажира - бинарный признак.</li>
<li>Parch - Количество родителей/детей на борту коробля - количественный признак.</li>
<li>IsChild - Является ли пассажир ребенком - бинарный признак.</li>
<li>AgeGroup - Возрастная группа пассажира - порядковый признак.</li>
</ul>

#### Оставим еще следующие признаки, которые, теоретически, должны улучшить результат:

<ul>
<li>SibSp - Количество братьев/сестер на борту коробля - количественный признак.</li>
<li>Fare - Цена билета - количественный признак.</li>
</ul>

### Для попробуем протестировать модель на тренировочных данных

#### Помимо лишних признаков я удаляю и PassengerId, т.к. он сильно ухудшает качество предсказания для kNN.

In [ ]:
X = df.drop(([
    'PassengerId', 
    'Survived', 
    'Name', 
    'Sex', 
    'Ticket',
    'Cabin',
    'Embarked',
    'Surname',
]), axis=1)

print(X.shape)
X.head(10)

Теперь необходимо сделать нормализацию данных, это необходимо как в случае с kNN, так и с логистической регрессией.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X

In [ ]:
y = df['Survived']
y.shape

#### kNN – метод ближайших соседей:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

<p>Поиск оптимальных значений параметров можно осуществить с помощью класса GridSearchCV – поиск наилучшего набора параметров, доставляющих минимум ошибке перекрестного контроля (cross-validation). По умолчанию рассматривается 3-кратный перекрестный контроль.</p>
<p>Найдем наилучшее значение k среди значений от 0 до 9:</p>

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
n_neighbors_array = list(range(1, 10))
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid={'n_neighbors': n_neighbors_array}, cv=10)
grid.fit(X_train, y_train)

print('grid.best_score_', grid.best_score_)
print('grid.best_params_', grid.best_params_)

In [ ]:
grid.score(X_test, y_test)

Как видно, наилучшее значение k равно 9, посчитаем accuracy_score для него.

#### Логистическая регрессия:

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
lr = LogisticRegressionCV(cv=10)
lr.fit(X_train, y_train)

In [ ]:
lr.scores_

In [ ]:
lr.scores_[1].mean(axis=0).max()

In [ ]:
lr.score(X_test, y_test)

Как видно, на этапе кросс валидации и в случае accuracy_score, kNN показывает лучшие результаты, чем логистическая регрессия. Поэтому оставим именно этот алгоритм.

### Теперь подготовим тестовую выборку

In [ ]:
df_test = df_test.fillna(df_test.median(axis=0), axis=0)

df_test['SexCode'] = df_test['Sex'].map(lambda sex: 1 if sex == 'male' else 0)

df_test['IsChild'] = df_test.apply(lambda row: 1 if row['Age'] <= 18 and row['Parch'] > 0 else 0, axis=1)

df_test['AgeGroup'] = df_test['Age']

df_test['AgeGroup'] = df_test['AgeGroup'].map(lambda age: int(age // 10) + 1)

df_test.head(5)

In [ ]:
X_prod = df_test.drop(([
    'PassengerId',  
    'Name', 
    'Sex', 
    'Ticket',
    'Cabin',
    'Embarked',
]), axis=1)

print(X_prod.shape)
X_prod.head(10)

In [ ]:
scaler = MinMaxScaler()
X_prod = scaler.fit_transform(X_prod)
X_prod

#### Создадим 2 предсказания для kNN и для логистической регрессии на случай, если первый покажет плохие результату на тестовых данных:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(X, y)

In [ ]:
y_pred = grid.predict(X_prod)

In [ ]:
y_pred

In [ ]:
df_predicted = pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Survived': y_pred})

In [ ]:
df_predicted.to_csv('sample_submission_knn.csv', sep=',', index=False)

#### Теперь тоже самое для логистической регрессии:

In [ ]:
lr = LogisticRegressionCV(cv=10)
lr.fit(X, y)

In [ ]:
y_pred = lr.predict(X_prod)

In [ ]:
y_pred

In [ ]:
df_predicted = pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Survived': y_pred})

In [ ]:
df_predicted.to_csv('sample_submission_lr.csv', sep=',', index=False)

В итоге, лучший результат для kaggle на текущий момент показан на логистической регрессии и равен 0.77511.